Take in SNPxSNP graph (from Cramer's V), and build graph.

In [2]:
def HL(dfMatrix) : 

    ## imports
    import numpy as np
    import networkx as nx
    import scipy.stats as stats
    import pandas as pd
    import sklearn as sk
    import itertools
    import os
    from bed_reader import open_bed
    from IPython.display import SVG
    from sknetwork.embedding import LouvainNE
    from sknetwork.embedding import LouvainEmbedding
    from sknetwork.visualization import svg_graph, svg_bigraph
    from networkx.algorithms import bipartite
    from networkx.algorithms import community
    from scipy import sparse
    from sknetwork.hierarchy import LouvainHierarchy
    from sknetwork.hierarchy import cut_straight, dasgupta_score, tree_sampling_divergence
    from sknetwork.visualization import svg_graph, svg_bigraph, svg_dendrogram
    from sklearn.neighbors import NearestNeighbors
    from scipy.stats import chi2_contingency
    from scipy.sparse import diags


    # binarize
    dfMatrix = sparse.csr_matrix(df) # convert data frame to a sparse matrix
    for i in range(np.shape(dfMatrix)[0]):
        for j in range(np.shape(dfMatrix)[1]):
            if dfMatrix[i,j] <= 0.3:
                dfMatrix[i,j] = 0
            else:
                dfMatrix[i,j] = 1


    # get triangle
    triangle = np.triu(dfMatrix, k=1) # get upper half
    print(triangle)
    print(triangle.shape)
    triangleSparse = sparse.csr_matrix(triangle)

    # create adjacency matrix 
    G = nx.Graph(triangleSparse) # create graph from sparse data [UPDATE THIS]
    adjacency = nx.adjacency_matrix(G) # create Adjacency matrix
    adjacencyMatrix = sparse.csr_matrix(adjacency) # convert array to sparse row matrix form
    print(adjacencyMatrix.shape[0])

    ## train classifier and build graph
    louvain = LouvainNE(2)
    embedding = louvain.fit_transform(adjacencyMatrix)
    degrees = adjacency.dot(np.ones(adjacency.shape[0]))

    labels = []
    for i in range(adjacencyMatrix.shape[0]):
        labels.append(i)

    image = svg_graph(adjacencyMatrix, position=embedding, display_edge_weight= True, display_node_weight=True, 
    labels=labels, scale=1, node_order=labels, filename="graph", node_size_min=10, node_size_max=20, directed=False)

    #result should be graph.svg file